# 🛒 Market Basket Analysis - Adım Adım Tutorial

Bu notebook, Market Basket Analysis'in temellerini adım adım öğretmek için hazırlanmıştır.

## 📚 Bu Tutorial'da Öğreneceğiniz Konular:
1. Market Basket Analysis nedir?
2. Support, Confidence, Lift kavramları
3. Veri yükleme ve hazırlama
4. Popüler ürünleri bulma
5. Ürün birlikteliklerini hesaplama
6. Association rules oluşturma
7. Sonuçları yorumlama
8. İş uygulamaları

## 1. 📖 Market Basket Analysis Nedir?

Market Basket Analysis (Market Sepeti Analizi), müşterilerin hangi ürünleri birlikte satın aldığını analiz eden bir veri madenciliği tekniğidir.

### 🎯 Amaçları:
- **Cross-selling**: Hangi ürünleri birlikte önerebiliriz?
- **Ürün yerleştirme**: Mağazada ürünleri nereye koymalıyız?
- **Kampanya planlama**: Hangi ürünlerde birlikte indirim yapalım?
- **Müşteri davranışı**: Müşteriler nasıl alışveriş yapıyor?

### 📊 Temel Kavramlar:
- **Support**: Bir ürün/ürün çiftinin ne kadar sık alındığı
- **Confidence**: A ürününü alanların kaçı B ürününü de alıyor
- **Lift**: İki ürünün birlikte alınma gücü

In [ ]:
# Gerekli kütüphaneleri yükleyin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Grafik ayarları
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ Kütüphaneler yüklendi!")

## 2. 📊 Veri Yükleme ve İnceleme

İlk olarak veriyi yükleyelim ve yapısını anlayalım.

In [ ]:
# Veriyi yükle
print("📁 Veri yükleniyor...")
veri = pd.read_csv('../data/basket_analysis.csv', index_col=0)

print(f"✅ Veri yüklendi!")
print(f"📊 Veri boyutu: {veri.shape[0]} sepet, {veri.shape[1]} ürün")
print(f"🏷️ Ürünler: {list(veri.columns)}")

In [ ]:
# İlk 5 satırı görelim
print("🔍 İlk 5 sepet:")
veri.head()

In [ ]:
# Veri tipi kontrolü
print("📋 Veri tipleri:")
print(veri.dtypes.unique())

print("\n🔢 Benzersiz değerler:")
print(veri.values.flatten()[:20])

### 💡 Veri Formatı Açıklaması:
- Her **satır** bir sepeti (transaction) temsil ediyor
- Her **sütun** bir ürünü temsil ediyor  
- **True/False** değerleri o ürünün sepette olup olmadığını gösteriyor
- Bu format **binary matrix** olarak adlandırılır

## 3. 🛍️ Sepetleri Hazırlama

Binary matrix formatındaki veriyi sepet listelerine çevirelim.

In [ ]:
# Sepetleri oluştur
def sepetleri_olustur(veri):
    sepetler = []
    
    for i, satir in veri.iterrows():
        sepet = []
        for urun in veri.columns:
            if satir[urun] == True or satir[urun] == 'True':
                sepet.append(urun)
        
        if sepet:  # Boş sepetleri ekleme
            sepetler.append(sepet)
    
    return sepetler

sepetler = sepetleri_olustur(veri)
print(f"✅ {len(sepetler)} sepet hazırlandı")

# İlk 5 sepeti görelim
print("\n🛒 İlk 5 sepet:")
for i, sepet in enumerate(sepetler[:5], 1):
    print(f"{i}. {sepet}")

In [ ]:
# Sepet istatistikleri
sepet_boyutlari = [len(sepet) for sepet in sepetler]

print("📊 SEPET İSTATİSTİKLERİ:")
print(f"🔸 Toplam sepet sayısı: {len(sepetler)}")
print(f"🔸 Ortalama ürün/sepet: {np.mean(sepet_boyutlari):.1f}")
print(f"🔸 En fazla ürün/sepet: {max(sepet_boyutlari)}")
print(f"🔸 En az ürün/sepet: {min(sepet_boyutlari)}")
print(f"🔸 Medyan ürün/sepet: {np.median(sepet_boyutlari):.1f}")

## 4. 📊 Popüler Ürünleri Bulma

Hangi ürünler en sık satın alınıyor?

In [ ]:
# Her ürünün kaç sepette olduğunu hesapla
def urun_popularitesi_hesapla(sepetler):
    urun_sayilari = {}
    
    for sepet in sepetler:
        for urun in sepet:
            urun_sayilari[urun] = urun_sayilari.get(urun, 0) + 1
    
    return urun_sayilari

urun_sayilari = urun_popularitesi_hesapla(sepetler)

# Popülarite sırasına göre sırala
sorted_urunler = sorted(urun_sayilari.items(), key=lambda x: x[1], reverse=True)

print("🏆 EN POPÜLER ÜRÜNLER:")
print("Sıra | Ürün        | Sepet Sayısı | Support (%)")
print("-" * 50)

for i, (urun, sayi) in enumerate(sorted_urunler, 1):
    support = (sayi / len(sepetler)) * 100
    print(f"{i:2d}.  | {urun:<12} | {sayi:8d}     | {support:6.1f}%")

In [ ]:
# Popülarite grafiği
plt.figure(figsize=(12, 8))

# En popüler 10 ürün
top_10 = sorted_urunler[:10]
urunler = [item[0] for item in top_10]
sayilar = [item[1] for item in top_10]

bars = plt.bar(urunler, sayilar, color='skyblue', alpha=0.8)
plt.title('En Popüler 10 Ürün', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Ürünler', fontsize=14)
plt.ylabel('Sepet Sayısı', fontsize=14)
plt.xticks(rotation=45, ha='right')

# Barların üzerine sayıları ekle
for bar, sayi in zip(bars, sayilar):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
             str(sayi), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.grid(axis='y', alpha=0.3)
plt.show()

print(f"📊 En popüler ürün: {sorted_urunler[0][0]} ({sorted_urunler[0][1]} sepet)")

## 5. 🔗 Support Kavramı

**Support**, bir itemset'in (ürün veya ürün grubu) ne kadar sık görüldüğünü ölçer.

### 📐 Formül:
```
Support(A) = A içeren sepet sayısı / Toplam sepet sayısı
```

### 💡 Yorumlama:
- Support ne kadar yüksekse, o itemset o kadar popülerdir
- Genellikle %5-10 ve üzeri support değerleri anlamlı kabul edilir

In [ ]:
# Support hesaplama örneği
def support_hesapla(urun, sepetler):
    urun_sayisi = sum(1 for sepet in sepetler if urun in sepet)
    support = urun_sayisi / len(sepetler)
    return support

# Örnek hesaplamalar
print("🧮 SUPPORT HESAPLAMA ÖRNEKLERİ:")
print("-" * 40)

ornek_urunler = ['chocolate', 'Milk', 'Bread', 'Apple']

for urun in ornek_urunler:
    support = support_hesapla(urun, sepetler)
    sepet_sayisi = sum(1 for sepet in sepetler if urun in sepet)
    
    print(f"📦 {urun}:")
    print(f"   Sepet sayısı: {sepet_sayisi}")
    print(f"   Support: {support:.3f} (%{support*100:.1f})")
    print()

## 6. 🤝 Ürün Birlikteliklerini Bulma

Hangi ürünler birlikte satın alınıyor? İki ürünlü kombinasyonları inceleyelim.

In [ ]:
def birliktelik_hesapla(sepetler, min_support=0.05):
    """İki ürünlü birliktelikleri hesaplar"""
    toplam_sepet = len(sepetler)
    min_sepet_sayisi = int(min_support * toplam_sepet)
    
    print(f"🔍 Minimum support: %{min_support*100:.0f} ({min_sepet_sayisi} sepet)")
    
    birliktelik_sayilari = {}
    
    # Her sepetteki ürün çiftlerini say
    for sepet in sepetler:
        if len(sepet) >= 2:
            # Bu sepetteki tüm ürün çiftleri
            for urun1, urun2 in combinations(sepet, 2):
                # Alfabetik sıra (tutarlılık için)
                if urun1 > urun2:
                    urun1, urun2 = urun2, urun1
                
                cift = (urun1, urun2)
                birliktelik_sayilari[cift] = birliktelik_sayilari.get(cift, 0) + 1
    
    # Minimum support'u geçenleri filtrele
    onemli_birliktelikler = {}
    for cift, sayi in birliktelik_sayilari.items():
        if sayi >= min_sepet_sayisi:
            support = sayi / toplam_sepet
            onemli_birliktelikler[cift] = {
                'sepet_sayisi': sayi,
                'support': support
            }
    
    return onemli_birliktelikler

# Birliktelikleri hesapla
birliktelikler = birliktelik_hesapla(sepetler, min_support=0.05)

print(f"\n✅ {len(birliktelikler)} önemli birliktelik bulundu!")

In [ ]:
# En güçlü birliktelikleri göster
sorted_birliktelikler = sorted(birliktelikler.items(), 
                              key=lambda x: x[1]['support'], reverse=True)

print("🤝 EN GÜÇLÜ BİRLİKTELİKLER:")
print("Sıra | Ürün Çifti                    | Sepet | Support")
print("-" * 60)

for i, (cift, bilgi) in enumerate(sorted_birliktelikler[:15], 1):
    urun1, urun2 = cift
    sepet_sayisi = bilgi['sepet_sayisi']
    support = bilgi['support']
    
    print(f"{i:2d}.  | {urun1:<12} + {urun2:<12} | {sepet_sayisi:3d}   | {support:5.1%}")

In [ ]:
# Birliktelik grafiği
plt.figure(figsize=(12, 8))

# En güçlü 10 birliktelik
top_10_birliktelik = sorted_birliktelikler[:10]

cift_isimleri = [f"{cift[0]} + {cift[1]}" for cift, _ in top_10_birliktelik]
sepet_sayilari = [bilgi['sepet_sayisi'] for _, bilgi in top_10_birliktelik]

bars = plt.barh(cift_isimleri, sepet_sayilari, color='lightcoral', alpha=0.8)
plt.title('En Güçlü 10 Ürün Birlikteliği', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Sepet Sayısı', fontsize=14)
plt.ylabel('Ürün Çiftleri', fontsize=14)

# Sayıları barların yanına ekle
for bar, sayi in zip(bars, sepet_sayilari):
    plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
             str(sayi), ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.grid(axis='x', alpha=0.3)
plt.show()

# En güçlü birliktelik
en_guclu = sorted_birliktelikler[0]
urun1, urun2 = en_guclu[0]
print(f"\n🏆 En güçlü birliktelik: {urun1} + {urun2} (%{en_guclu[1]['support']*100:.1f})")

## 7. 📋 Confidence Kavramı

**Confidence**, "A ürününü alan müşterilerin kaçta kaçının B ürününü de aldığını" ölçer.

### 📐 Formül:
```
Confidence(A → B) = Support(A,B) / Support(A)
```

### 💡 Yorumlama:
- %50+ confidence genellikle anlamlı kabul edilir
- "A alan müşterilerin %X'i B'yi de alıyor" şeklinde yorumlanır

In [ ]:
def kural_olustur(birliktelikler, urun_sayilari, toplam_sepet, min_confidence=0.3):
    """Association rules oluşturur"""
    kurallar = []
    
    for (urun1, urun2), bilgi in birliktelikler.items():
        birlikte_sayi = bilgi['sepet_sayisi']
        
        # Kural 1: urun1 → urun2
        confidence1 = birlikte_sayi / urun_sayilari[urun1]
        if confidence1 >= min_confidence:
            lift1 = confidence1 / (urun_sayilari[urun2] / toplam_sepet)
            kurallar.append({
                'antecedent': urun1,      # Öncül (A)
                'consequent': urun2,      # Sonuç (B) 
                'support': bilgi['support'],
                'confidence': confidence1,
                'lift': lift1
            })
        
        # Kural 2: urun2 → urun1
        confidence2 = birlikte_sayi / urun_sayilari[urun2]
        if confidence2 >= min_confidence:
            lift2 = confidence2 / (urun_sayilari[urun1] / toplam_sepet)
            kurallar.append({
                'antecedent': urun2,
                'consequent': urun1,
                'support': bilgi['support'],
                'confidence': confidence2,
                'lift': lift2
            })
    
    return sorted(kurallar, key=lambda x: x['confidence'], reverse=True)

# Kuralları oluştur
kurallar = kural_olustur(birliktelikler, urun_sayilari, len(sepetler), min_confidence=0.3)

print(f"🔍 Minimum confidence: %30")
print(f"✅ {len(kurallar)} association rule bulundu!")

In [ ]:
# En güçlü kuralları göster
print("📋 EN GÜÇLÜ ASSOCIATION RULES:")
print("Sıra | Öncül       → Sonuç       | Support | Confidence | Lift")
print("-" * 70)

for i, kural in enumerate(kurallar[:15], 1):
    print(f"{i:2d}.  | {kural['antecedent']:<12} → {kural['consequent']:<12} | "
          f"{kural['support']:5.1%}   | {kural['confidence']:8.1%}   | {kural['lift']:4.2f}")

### 💡 Confidence Açıklaması:

Yukarıdaki ilk kural için örnek:
- **Milk → chocolate**: %68 confidence
- **Anlamı**: Milk alan müşterilerin %68'i chocolate da alıyor
- **İş uygulaması**: Milk alan müşterilere chocolate önerebiliriz

## 8. 🚀 Lift Kavramı

**Lift**, iki ürünün birlikte alınma olasılığının tesadüfi duruma göre ne kadar güçlü olduğunu ölçer.

### 📐 Formül:
```
Lift(A → B) = Confidence(A → B) / Support(B)
```

### 💡 Yorumlama:
- **Lift = 1**: Tesadüfi birliktelik (bağımsız)
- **Lift > 1**: Pozitif birliktelik (birlikte alınma eğilimi var)
- **Lift < 1**: Negatif birliktelik (birlikte alınma eğilimi yok)
- **Lift > 1.5**: Güçlü pozitif birliktelik

In [ ]:
# Lift değerlerini analiz et
lift_degerleri = [kural['lift'] for kural in kurallar]

print("🚀 LIFT ANALİZİ:")
print(f"🔸 Ortalama Lift: {np.mean(lift_degerleri):.2f}")
print(f"🔸 Maksimum Lift: {max(lift_degerleri):.2f}")
print(f"🔸 Minimum Lift: {min(lift_degerleri):.2f}")

# Lift kategorileri
guclu_kurallar = [k for k in kurallar if k['lift'] > 1.5]
orta_kurallar = [k for k in kurallar if 1.0 < k['lift'] <= 1.5]
zayif_kurallar = [k for k in kurallar if k['lift'] <= 1.0]

print(f"\n📊 LIFT KATEGORİLERİ:")
print(f"🟢 Güçlü kurallar (Lift > 1.5): {len(guclu_kurallar)}")
print(f"🟡 Orta kurallar (1.0 < Lift ≤ 1.5): {len(orta_kurallar)}")
print(f"🔴 Zayıf kurallar (Lift ≤ 1.0): {len(zayif_kurallar)}")

In [ ]:
# Lift histogramı
plt.figure(figsize=(10, 6))

plt.hist(lift_degerleri, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Lift = 1 (Tesadüfi)')
plt.axvline(x=1.5, color='orange', linestyle='--', linewidth=2, label='Lift = 1.5 (Güçlü)')

plt.title('Lift Değerlerinin Dağılımı', fontsize=16, fontweight='bold')
plt.xlabel('Lift Değeri', fontsize=12)
plt.ylabel('Kural Sayısı', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"📊 Lift grafiği: Çoğu kural 1.0'dan büyük lift değerine sahip (pozitif birliktelik)")

In [ ]:
# En güçlü kuralların detaylı açıklaması
print("🏆 EN GÜÇLÜ 3 KURALIN DETAYLI AÇIKLAMASI:")
print("=" * 60)

for i, kural in enumerate(kurallar[:3], 1):
    oncul = kural['antecedent']
    sonuc = kural['consequent']
    confidence = kural['confidence']
    lift = kural['lift']
    support = kural['support']
    
    print(f"\n{i}. KURAL: {oncul} → {sonuc}")
    print(f"   📊 Support: %{support*100:.1f} (Sepetlerin %{support*100:.1f}'inde birlikte görülüyor)")
    print(f"   🎯 Confidence: %{confidence*100:.1f} ({oncul} alan müşterilerin %{confidence*100:.0f}'i {sonuc} da alıyor)")
    print(f"   🚀 Lift: {lift:.2f} (Tesadüften {lift:.1f} kat daha güçlü birliktelik)")
    
    print(f"   💡 İş Uygulaması:")
    print(f"      - {oncul} alan müşterilere {sonuc} önerebilirsiniz")
    print(f"      - Bu ürünleri mağazada yakın yerlere koyabilirsiniz")
    print(f"      - {oncul} + {sonuc} için birlikte kampanya yapabilirsiniz")

## 9. 🎯 Ürün Önerisi Sistemi

Artık bir ürün için öneriler üretebiliriz!

In [ ]:
def urun_onerisi_uret(secilen_urun, kurallar, top_n=5):
    """Belirli bir ürün için öneriler üretir"""
    
    # Bu ürün için geçerli kuralları bul
    uygun_kurallar = [kural for kural in kurallar 
                     if kural['antecedent'] == secilen_urun]
    
    if not uygun_kurallar:
        return []
    
    # Confidence'a göre sırala
    uygun_kurallar.sort(key=lambda x: x['confidence'], reverse=True)
    
    return uygun_kurallar[:top_n]

# Örnek öneriler
ornek_urunler = ['Milk', 'Bread', 'chocolate', 'Apple']

for urun in ornek_urunler:
    print(f"\n🎯 '{urun}' İÇİN ÖNERİLER:")
    print("-" * 40)
    
    oneriler = urun_onerisi_uret(urun, kurallar, top_n=3)
    
    if oneriler:
        for i, kural in enumerate(oneriler, 1):
            print(f"{i}. {kural['consequent']}")
            print(f"   Güven: %{kural['confidence']*100:.1f}")
            print(f"   Lift: {kural['lift']:.2f}")
            print(f"   Açıklama: '{urun}' alan müşterilerin %{kural['confidence']*100:.0f}'i '{kural['consequent']}' da alıyor")
            print()
    else:
        print(f"   ❌ '{urun}' için öneri bulunamadı")
        print(f"   Bu ürün başka ürünlerle güçlü birliktelik oluşturmuyor")

In [ ]:
# İnteraktif öneri sistemi
def onerileri_gorsellestir(secilen_urun, kurallar):
    """Önerileri grafik olarak gösterir"""
    
    oneriler = urun_onerisi_uret(secilen_urun, kurallar, top_n=5)
    
    if not oneriler:
        print(f"❌ '{secilen_urun}' için öneri bulunamadı")
        return
    
    # Grafik verilerini hazırla
    urun_isimleri = [kural['consequent'] for kural in oneriler]
    confidence_skorlari = [kural['confidence']*100 for kural in oneriler]
    
    # Grafik çiz
    plt.figure(figsize=(10, 6))
    bars = plt.barh(urun_isimleri, confidence_skorlari, color='lightblue', alpha=0.8)
    
    plt.title(f"'{secilen_urun}' için Ürün Önerileri", fontsize=16, fontweight='bold')
    plt.xlabel('Confidence Skoru (%)', fontsize=12)
    plt.ylabel('Önerilen Ürünler', fontsize=12)
    
    # Skorları barların yanına ekle
    for bar, skor in zip(bars, confidence_skorlari):
        plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                 f'{skor:.1f}%', ha='left', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.grid(axis='x', alpha=0.3)
    plt.show()
    
    return oneriler

# Milk için önerileri görselleştir
print("📊 Milk için öneriler:")
milk_onerileri = onerileri_gorsellestir('Milk', kurallar)

## 10. 📊 Sonuçların Özeti ve İş Uygulamaları

In [ ]:
# Genel özet raporu
print("📋 MARKET BASKET ANALİZİ ÖZET RAPORU")
print("=" * 50)

print(f"📊 VERİ ÖZETİ:")
print(f"   🔸 Toplam sepet sayısı: {len(sepetler):,}")
print(f"   🔸 Toplam ürün çeşidi: {len(urun_sayilari)}")
print(f"   🔸 Ortalama ürün/sepet: {np.mean(sepet_boyutlari):.1f}")

print(f"\n🤝 BİRLİKTELİK ANALİZİ:")
print(f"   🔸 Bulunan ürün çifti birlikteliği: {len(birliktelikler)}")
print(f"   🔸 Minimum support eşiği: %5")

print(f"\n📋 ASSOCIATION RULES:")
print(f"   🔸 Toplam kural sayısı: {len(kurallar)}")
print(f"   🔸 Minimum confidence eşiği: %30")
print(f"   🔸 Güçlü kurallar (Lift > 1.5): {len(guclu_kurallar)}")

# En popüler ürün
en_populer_urun, en_populer_sayi = sorted_urunler[0]
print(f"\n🏆 EN POPÜLER ÜRÜN:")
print(f"   {en_populer_urun}: {en_populer_sayi} sepet (%{(en_populer_sayi/len(sepetler))*100:.1f})")

# En güçlü birliktelik
en_guclu_cift, en_guclu_bilgi = sorted_birliktelikler[0]
print(f"\n🤝 EN GÜÇLÜ BİRLİKTELİK:")
print(f"   {en_guclu_cift[0]} + {en_guclu_cift[1]}: %{en_guclu_bilgi['support']*100:.1f} support")

# En güçlü kural
en_guclu_kural = kurallar[0]
print(f"\n📋 EN GÜÇLÜ KURAL:")
print(f"   {en_guclu_kural['antecedent']} → {en_guclu_kural['consequent']}: %{en_guclu_kural['confidence']*100:.1f} confidence")

In [ ]:
# İş önerileri
print("\n💼 İŞ ÖNERİLERİ VE UYGULAMALAR:")
print("=" * 50)

print("🛒 CROSS-SELLING (ÇAPRAZ SATIŞ):")
for i, kural in enumerate(kurallar[:3], 1):
    print(f"   {i}. {kural['antecedent']} alan müşterilere {kural['consequent']} önerin")
    print(f"      (%{kural['confidence']*100:.0f} başarı oranı beklenir)")

print(f"\n🏪 MAĞAZA DÜZENİ:")
for i, (cift, bilgi) in enumerate(sorted_birliktelikler[:3], 1):
    urun1, urun2 = cift
    print(f"   {i}. {urun1} ve {urun2}'ı yakın raflara koyun")
    print(f"      (Sepetlerin %{bilgi['support']*100:.1f}'inde birlikte alınıyor)")

print(f"\n🎯 KAMPANYA PLANLAMA:")
print(f"   1. En popüler ürün {en_populer_urun} için özel kampanya")
print(f"   2. {en_guclu_cift[0]} + {en_guclu_cift[1]} kombo paketi oluşturun")
print(f"   3. Güçlü kurallar için '2. ürün %50 indirim' kampanyası")

print(f"\n📊 STOK YÖNETİMİ:")
print(f"   1. Birlikte alınan ürünlerin stoklarını birlikte planlayın")
print(f"   2. {en_populer_urun} bitiyorsa, birlikte alınan ürünler de bitebilir")
print(f"   3. Mevsimsel değişiklikler için bu analizleri tekrarlayın")

## 11. 🎓 Öğrenim Değerlendirmesi

### ✅ Bu Tutorial'da Öğrendikleriniz:

1. **Market Basket Analysis'in temel kavramları**
   - Support: Popülarite ölçüsü
   - Confidence: Koşullu olasılık
   - Lift: Birliktelik gücü

2. **Veri hazırlama ve analiz süreci**
   - Binary matrix formatından sepet listelerine dönüşüm
   - Ürün popülaritesi hesaplama
   - Birliktelik matrisi oluşturma

3. **Association Rules oluşturma**
   - İki yönlü kurallar (A→B ve B→A)
   - Threshold belirleme (min_support, min_confidence)
   - Kural kalitesi değerlendirme

4. **Sonuçları yorumlama ve iş uygulamaları**
   - Cross-selling stratejileri
   - Mağaza düzeni optimizasyonu
   - Kampanya planlama
   - Stok yönetimi

### 🚀 Sonraki Adımlar:
- Daha büyük dataset'lerle çalışma
- 3+ ürünlü kombinasyonları analiz etme
- Zaman bazlı analiz (hangi ürünler hangi dönemlerde birlikte alınıyor?)
- Müşteri segmentasyonu ile birleştirme
- Gerçek e-ticaret verisi ile uygulama

In [ ]:
# Tutorial tamamlandı mesajı
print("🎉 TUTORIAL TAMAMLANDI! 🎉")
print("="*40)
print("✅ Market Basket Analysis temellerini öğrendiniz")
print("✅ Support, Confidence, Lift kavramlarını anladınız")
print("✅ Association Rules oluşturmayı öğrendiniz")
print("✅ Sonuçları iş uygulamalarına dönüştürmeyi öğrendiniz")
print("")
print("🚀 Artık kendi verilerinizle analiz yapabilirsiniz!")
print("📚 Daha ileri konular için ek kaynakları inceleyin")
print("💡 Sorularınız için eğitmeninizle iletişime geçin")